In [14]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html

### Q1

In [1]:
# https://stackoverflow.com/questions/51079543/pandas-groupby-apply-vs-transform-with-specific-functions


# groupby 
# https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html

In [31]:
import pandas as pd
import numpy as np
from IPython.display import display

In [3]:
df = pd.DataFrame({'a': [1,2,3,1,2,3,3], 'b':[1,2,3,1,2,3,3], 'type':[1,2,3,4,5,6,6]})

In [4]:
df

,a,b,type
0,1,1,1
1,2,2,2
2,3,3,3
3,1,1,4
4,2,2,5
5,3,3,6
6,3,3,6


In [61]:
df['g'] = df.groupby(['a', 'b'])['type'].transform(lambda x: np.array(set(x)))
print(df)

KeyError: 'Column not found: type'

In [6]:
pd.__version__

'1.3.4'

### Q2

In [7]:
# https://stackoverflow.com/questions/27517425/apply-vs-transform-on-a-group-object

In [8]:
columns = ['A', 'B', 'C', 'D']
records = [
    ['foo', 'one', 0.162003, 0.087469],
    ['bar', 'one', -1.156319, -1.5262719999999999],
    ['foo', 'two', 0.833892, -1.666304],     
    ['bar', 'three', -2.026673, -0.32205700000000004],
    ['foo', 'two', 0.41145200000000004, -0.9543709999999999],
    ['bar', 'two', 0.765878, -0.095968],
    ['foo', 'one', -0.65489, 0.678091],
    ['foo', 'three', -1.789842, -1.130922]
]
df = pd.DataFrame.from_records(records, columns=columns)

In [9]:
df

,A,B,C,D
0,foo,one,0.162003,0.087469
1,bar,one,-1.156319,-1.526272
2,foo,two,0.833892,-1.666304
3,bar,three,-2.026673,-0.322057
4,foo,two,0.411452,-0.954371
5,bar,two,0.765878,-0.095968
6,foo,one,-0.654890,0.678091
7,foo,three,-1.789842,-1.130922


In [10]:
df.groupby('A').apply(lambda x: (x['C'] - x['D']))

A     
bar  1    0.369953
     3   -1.704616
     5    0.861846
foo  0    0.074534
     2    2.500196
     4    1.365823
     6   -1.332981
     7   -0.658920
dtype: float64

In [11]:
df.groupby('A').apply(lambda x: (x['C'] - x['D']).mean())

A
bar   -0.157606
foo    0.389730
dtype: float64

In [12]:
df.groupby('A').transform(lambda x: (x['C'] - x['D']))

KeyError: 'C'

In [13]:
df.groupby('A').transform(lambda x: (x['C'] - x['D']).mean())

KeyError: 'C'

In [15]:
zscore = lambda x: (x - x.mean()) / x.std()

In [16]:
zscore

<function __main__.<lambda>(x)>

In [18]:
transformed = df.groupby(key).transform(zscore)

NameError: name 'key' is not defined

In [20]:
df = pd.DataFrame({'State':['Texas', 'Texas', 'Florida', 'Florida'], 
                   'a':[4,5,1,3], 'b':[6,10,3,11]})

In [21]:
df

,State,a,b
0,Texas,4,6
1,Texas,5,10
2,Florida,1,3
3,Florida,3,11


In [22]:
def inspect(x):
    print(type(x))
    raise

# apply --> takes dataframe as input

In [23]:
df.groupby('State').apply(inspect)

<class 'pandas.core.frame.DataFrame'>


RuntimeError: No active exception to reraise

# transform --> takes column as input

In [24]:
df.groupby('State').transform(inspect)

<class 'pandas.core.series.Series'>


RuntimeError: No active exception to reraise

In [32]:
def subtract_two(x):
    display(x)
    return x['a'] - x['b']

In [29]:
df.groupby('State').transform(subtract_two) # fails

KeyError: 'a'

In [34]:
df

,State,a,b
0,Texas,4,6
1,Texas,5,10
2,Florida,1,3
3,Florida,3,11


In [36]:
df.groupby('State').apply(subtract_two) #.reset_index() # executes

,State,a,b
2,Florida,1,3
3,Florida,3,11


,State,a,b
0,Texas,4,6
1,Texas,5,10


State     
Florida  2   -2
         3   -8
Texas    0   -2
         1   -5
dtype: int64

In [38]:
# Transform must return a single dimensional sequence the same size as the group

In [42]:
def return_three(x):
    return np.array([1, 2, 3])

In [43]:
df.groupby('State').transform(return_three)

ValueError: Length mismatch: Expected axis has 6 elements, new values have 4 elements

In [49]:
df

,State,a,b
0,Texas,4,6
1,Texas,5,10
2,Florida,1,3
3,Florida,3,11


In [52]:
np.random.rand(2)

array([0.0388946 , 0.40263232])

In [47]:
def rand_group_len(x):
    display(x)
    display(np.random.rand(len(x)))
    return np.random.rand(len(x))

In [54]:
df.groupby('State', sort=False).transform(rand_group_len)

0    4
1    5
Name: a, dtype: int64

array([0.96070602, 0.60484916])

0     6
1    10
Name: b, dtype: int64

array([0.38033225, 0.36344283])

,a,b
0,4,6
1,5,10


...

2     3
3    11
Name: b, dtype: int64

array([0.3913553 , 0.85501706])

,a,b
2,1,3
3,3,11


array([0.41102495, 0.69411751])

,a,b
0,0.707195,0.412260
1,0.279514,0.047354
2,0.621682,0.516495
3,0.904611,0.447534


In [55]:
df.groupby('State', sort=False)['a'].transform(rand_group_len)

0    4
1    5
Name: Texas, dtype: int64

array([0.84823024, 0.66633211])

2    1
3    3
Name: Florida, dtype: int64

array([0.34541864, 0.39324023])

0    0.610013
1    0.567424
2    0.775157
3    0.531742
Name: a, dtype: float64

In [56]:
# Returning a single scalar object also works for transform

In [57]:
def group_sum(x):
    return x.sum()

In [58]:
df.groupby('State').transform(group_sum)

,a,b
0,9,16
1,9,16
2,4,14
3,4,14


## Q3

In [59]:
# https://stackoverflow.com/questions/13854476/pandas-transform-doesnt-work-sorting-groupby-output/13854901#13854901